# Blockbusters

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import csv

In [2]:
link = "https://www.boxofficemojo.com/year/1980/"
response = requests.get(link)
html = response.text

soup = BeautifulSoup(html)

In [3]:
header = ["Year", "Rank", "Release", "Gross", "Theaters", "Total Gross", "Release Date", "Distributor"]
years = range(1980, 2023) # years from 1980 - 2022, inclusive
movies = []

In [4]:
years = range(1980, 2023) # years from 1980 - 2022, inclusive
movies = []

for year in years:
    link = f"https://www.boxofficemojo.com/year/{year}/"
    response = requests.get(link)
    html = response.text

    soup = BeautifulSoup(html)

    tr_elements = soup.select("table tr")[1:]

    for tr in tr_elements[:100]:
        for index, td in enumerate(tr.select("td")):
            if index == 0:
                rank = td.text.strip()
            elif index == 1:
                release = td.text.strip()
            elif index == 5:
                gross = td.text.replace('$', '').replace(',', '').strip()
            elif index == 6:
                theaters = td.text.replace(',', '').strip()
            elif index == 7:
                total_gross = td.text.replace('$', '').replace(',', '').strip()
            elif index == 8:
                release_date = td.text.strip()
            elif index == 9:
                distributor = td.text.strip()


        movie_info = {
            'year': year,
            'rank': rank,
            'release': release,
            'gross': gross,
            'theaters': theaters,
            'total_gross': total_gross,
            'release_date': release_date,
            'distributor': distributor,
        }

        movies.append(movie_info)

    

In [5]:
df = pd.DataFrame(movies)

In [6]:
df['release_date_formatted'] =  df['release_date'] + ' ' + df['year'].astype(str)
df['release_date_formatted'] = pd.to_datetime(df['release_date_formatted'], format="%b %d %Y", errors='coerce').dt.date

df

,year,rank,release,gross,theaters,total_gross,release_date,distributor,release_date_formatted
0,1980,1,Star Wars: Episode V - The Empire Strikes Back,203359628,1278,209398025,May 21,Twentieth Century Fox,1980-05-21
1,1980,2,9 to 5,103290500,910,103290500,Dec 19,Twentieth Century Fox,1980-12-19
2,1980,3,Stir Crazy,101300000,813,101300000,Dec 12,Columbia Pictures,1980-12-12
3,1980,4,Kramer vs. Kramer,94448310,898,106260000,Dec 19,Columbia Pictures,1980-12-19
4,1980,5,Any Which Way You Can,70687344,1541,70687344,Dec 17,Warner Bros.,1980-12-17
...,...,...,...,...,...,...,...,...,...
4219,2022,96,I Heard the Bells,5563323,1204,5644104,Dec 1,Fathom Events,2022-12-01
4220,2022,97,Encanto,5465293,3980,96093622,Nov 24,Walt Disney Studios Motion Pictures,2022-11-24
4221,2022,98,Orphan: First Kill,5413227,557,5413227,Aug 19,Paramount Pictures,2022-08-19
4222,2022,99,House of Gucci,5392076,3477,53809574,Nov 24,United Artists Releasing,2022-11-24


### seasons
based on <a href="https://www.boxofficemojo.com/season/summer/?grossesOption=calendarGrosses">boxofficemojo's</a> designations
<ul><li>winter is jan 6 - march 2</li>
<li>spring is march 3 - may 4</li>
<li>summer is may 5 - sep 4</li>
<li>fall is sep 5 - nov 2</li>
<li>holiday season is nov 3 - jan 5</li></ul>

In [7]:
df['seasons'] = df['release_date_formatted'].apply(
    lambda x: 'winter' if pd.notnull(x) and '01-06' <= x.strftime('%m-%d') <= '03-02' else
    'spring' if pd.notnull(x) and '03-03' <= x.strftime('%m-%d') <= '05-04' else
    'summer' if pd.notnull(x) and '05-05' <= x.strftime('%m-%d') <= '09-04' else
    'fall' if pd.notnull(x) and '09-05' <= x.strftime('%m-%d') <= '11-02' else
    'holiday season' if pd.notnull(x) and ('11-03' <= x.strftime('%m-%d') <= '12-31' or x.strftime('%m-%d') <= '01-05') else
    None
)
df

,year,rank,release,gross,theaters,total_gross,release_date,distributor,release_date_formatted,seasons
0,1980,1,Star Wars: Episode V - The Empire Strikes Back,203359628,1278,209398025,May 21,Twentieth Century Fox,1980-05-21,summer
1,1980,2,9 to 5,103290500,910,103290500,Dec 19,Twentieth Century Fox,1980-12-19,holiday season
2,1980,3,Stir Crazy,101300000,813,101300000,Dec 12,Columbia Pictures,1980-12-12,holiday season
3,1980,4,Kramer vs. Kramer,94448310,898,106260000,Dec 19,Columbia Pictures,1980-12-19,holiday season
4,1980,5,Any Which Way You Can,70687344,1541,70687344,Dec 17,Warner Bros.,1980-12-17,holiday season
...,...,...,...,...,...,...,...,...,...,...
4219,2022,96,I Heard the Bells,5563323,1204,5644104,Dec 1,Fathom Events,2022-12-01,holiday season
4220,2022,97,Encanto,5465293,3980,96093622,Nov 24,Walt Disney Studios Motion Pictures,2022-11-24,holiday season
4221,2022,98,Orphan: First Kill,5413227,557,5413227,Aug 19,Paramount Pictures,2022-08-19,summer
4222,2022,99,House of Gucci,5392076,3477,53809574,Nov 24,United Artists Releasing,2022-11-24,holiday season


adjust the gross

In [8]:
df['adjusted_gross'] = pd.NA
df['adjusted_total_gross'] = pd.NA

# move the column next to gross
adjusted_gross_index = df.columns.get_loc('adjusted_gross')
df.insert(4, 'adjusted_gross', df.pop('adjusted_gross'))

# move the column next to total_gross
adjusted_total_gross_index = df.columns.get_loc('adjusted_total_gross')
df.insert(7, 'adjusted_total_gross', df.pop('adjusted_total_gross'))

df

,year,rank,release,gross,adjusted_gross,theaters,total_gross,adjusted_total_gross,release_date,distributor,release_date_formatted,seasons
0,1980,1,Star Wars: Episode V - The Empire Strikes Back,203359628,NaN,1278,209398025,NaN,May 21,Twentieth Century Fox,1980-05-21,summer
1,1980,2,9 to 5,103290500,NaN,910,103290500,NaN,Dec 19,Twentieth Century Fox,1980-12-19,holiday season
2,1980,3,Stir Crazy,101300000,NaN,813,101300000,NaN,Dec 12,Columbia Pictures,1980-12-12,holiday season
3,1980,4,Kramer vs. Kramer,94448310,NaN,898,106260000,NaN,Dec 19,Columbia Pictures,1980-12-19,holiday season
4,1980,5,Any Which Way You Can,70687344,NaN,1541,70687344,NaN,Dec 17,Warner Bros.,1980-12-17,holiday season
...,...,...,...,...,...,...,...,...,...,...,...,...
4219,2022,96,I Heard the Bells,5563323,NaN,1204,5644104,NaN,Dec 1,Fathom Events,2022-12-01,holiday season
4220,2022,97,Encanto,5465293,NaN,3980,96093622,NaN,Nov 24,Walt Disney Studios Motion Pictures,2022-11-24,holiday season
4221,2022,98,Orphan: First Kill,5413227,NaN,557,5413227,NaN,Aug 19,Paramount Pictures,2022-08-19,summer
4222,2022,99,House of Gucci,5392076,NaN,3477,53809574,NaN,Nov 24,United Artists Releasing,2022-11-24,holiday season


In [9]:
# https://data.bls.gov/cgi-bin/cpicalc.pl

inflation_rates = {
    1980: 3.85,
    1981: 3.44,
    1982: 3.17,
    1983: 3.06,
    1984: 2.94,
    1985: 2.84,
    1986: 2.73,
    1987: 2.69,
    1988: 2.59,
    1989: 2.47,
    1990: 2.35,
    1991: 2.22,
    1992: 2.17,
    1993: 2.10,
    1994: 2.05,
    1995: 1.99,
    1996: 1.94,
    1997: 1.88,
    1998: 1.85,
    1999: 1.82,
    2000: 1.77,
    2001: 1.71,
    2002: 1.69,
    2003: 1.65,
    2004: 1.62,
    2005: 1.57,
    2006: 1.51,
    2007: 1.48,
    2008: 1.42,
    2009: 1.42,
    2010: 1.38,
    2011: 1.36, 
    2012: 1.32,
    2013: 1.30,
    2014: 1.28,
    2015: 1.28,
    2016: 1.26,
    2017: 1.23,
    2018: 1.21,
    2019: 1.19,
    2020: 1.16,
    2021: 1.14,
    2022: 1.06
}

In [10]:
df['gross'] = pd.to_numeric(df['gross'], errors='coerce')
df['total_gross'] = pd.to_numeric(df['total_gross'], errors='coerce')

for year in years:
    df_year = df[df['year'] == year]

    df.loc[df_year.index, 'adjusted_gross'] = df_year['gross'] * inflation_rates[year]

    df.loc[df_year.index, 'adjusted_total_gross'] = df_year['total_gross'] * inflation_rates[year]

df['adjusted_gross'] = df['adjusted_gross'].astype(int)
df['adjusted_total_gross'] = df['adjusted_total_gross'].astype(int)

df

,year,rank,release,gross,adjusted_gross,theaters,total_gross,adjusted_total_gross,release_date,distributor,release_date_formatted,seasons
0,1980,1,Star Wars: Episode V - The Empire Strikes Back,203359628,782934567,1278,209398025,806182396,May 21,Twentieth Century Fox,1980-05-21,summer
1,1980,2,9 to 5,103290500,397668425,910,103290500,397668425,Dec 19,Twentieth Century Fox,1980-12-19,holiday season
2,1980,3,Stir Crazy,101300000,390005000,813,101300000,390005000,Dec 12,Columbia Pictures,1980-12-12,holiday season
3,1980,4,Kramer vs. Kramer,94448310,363625993,898,106260000,409101000,Dec 19,Columbia Pictures,1980-12-19,holiday season
4,1980,5,Any Which Way You Can,70687344,272146274,1541,70687344,272146274,Dec 17,Warner Bros.,1980-12-17,holiday season
...,...,...,...,...,...,...,...,...,...,...,...,...
4219,2022,96,I Heard the Bells,5563323,5897122,1204,5644104,5982750,Dec 1,Fathom Events,2022-12-01,holiday season
4220,2022,97,Encanto,5465293,5793210,3980,96093622,101859239,Nov 24,Walt Disney Studios Motion Pictures,2022-11-24,holiday season
4221,2022,98,Orphan: First Kill,5413227,5738020,557,5413227,5738020,Aug 19,Paramount Pictures,2022-08-19,summer
4222,2022,99,House of Gucci,5392076,5715600,3477,53809574,57038148,Nov 24,United Artists Releasing,2022-11-24,holiday season


In [13]:
df.to_csv('movies.csv', index=False)

# sum of gross

In [21]:
df_gross = df.groupby(['release', 'distributor', 'seasons'])[['adjusted_gross']].sum().reset_index()

df_gross

,release,distributor,seasons,adjusted_gross
0,'83,Reliance Entertainment,holiday season,3448423
1,*batteries not included,Universal Pictures,holiday season,86795885
2,10 Cloverfield Lane,Paramount Pictures,spring,90824577
3,10 Things I Hate About You,Walt Disney Studios Motion Pictures,spring,69484262
4,10 to Midnight,Metro-Goldwyn-Mayer (MGM),spring,21957311
...,...,...,...,...
3885,Zoot Suit,Universal Pictures,fall,11200922
3886,Zootopia,Walt Disney Studios Motion Pictures,spring,429997992
3887,xXx,Revolution Studios,summer,240069063
3888,xXx: Return of Xander Cage,Paramount Pictures,winter,55225047


In [22]:
df_gross.to_csv('movies_gross.csv', index=False)